In [84]:
import numpy as np 
import pandas as pd 

import inspect
from IPython.display import display, HTML

# Combinación de varios DataFrames

Vamos a definir una función que nos permitirá crear los DataFrame de cualquier dimension de forma rápida para poder realizar las pruebas

In [164]:
def show_dfs(*dfs, caption=""):
    '''
    Funcion de ayuda para mostrar varios dataframes en la 
    misma fila.
    
    dfs: varios DataFrames
    '''
    res = """<table border="4" class="dataframe">
             <caption style="font-size:1.2em;">
                 <strong>{0}</strong>
             </caption>""".format(caption)
    res += '<tr>'
    for t in dfs:
        res += '<td style="border: 1px solid black; vertical-align:top;">'+t.to_html()+'</td>'
    res += '</tr></table>'
    display(HTML(res))

In [165]:
def make_df (index=range(2), columns=list('AB'), prefix_value = ""):
    datos = {
        c:[prefix_value+str(c)+str(i) for i in index]
        for c in columns
    }
    return pd.DataFrame(datos, index)

make_df()

,A,B
0,A0,B0
1,A1,B1


## Concatenación

In [166]:
a = make_df (index=[0,1])
b = make_df (index=[2,3])
show_dfs(a,b,caption="DF's de prueba")

,A,B
0,A0,B0
1,A1,B1
,A,B
2,A2,B2
3,A3,B3


`concat` es un método de Pandas que permite unir DataFrames agregándolos como nuevas filas o como nuevas columnas.  
A continuación un ejemplo de agregación como nuevas filas

In [88]:
show_dfs(a,b,pd.concat([a,b])) #Añade filas

,A,B
0,A0,B0
1,A1,B1
,A,B
2,A2,B2
3,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


La unión de las filas de varios DataFrames tiene un método más directo, `append`. En la siguiente celda se ve como se realiza.  
Si se va a realizar la unión de varios DataFrames, es más óptimo crear una lista con todos los dataframes a unir y utilizar el método `pandas.concat()`

In [89]:
a.append(b) 

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


También tenemos la opción de unir a un DataFrame las columnas de otro DataFrame, utilizaremos la opción `axis=1` o `axis="columns"` del método `concat`

In [92]:
b = make_df (columns=list('CD'))
show_dfs(a,b,pd.concat([a,b],axis='columns')) #Añade columnas

En algunos casos la concatenación  puede generar índices o columnas duplicadas. Esta situación se puede resolver con la opción `ignore_index`, que recalcula los índices en el DataFrame resultante.  
*Si queremos controlar que no se produzcan repetidos podemos utilizar el parámetro `pandas.concat( ..., verify_integrity = True)` que genera un error cuando hay índices duplicados en los DataFrames*

In [94]:
b = make_df(prefix_value='B')
show_dfs (pd.concat([a,b]), 
          pd.concat([a,b],ignore_index=True))

,A,B
0,A0,B0
1,A1,B1
0,BA0,BB0
1,BA1,BB1
,A,B
0,A0,B0
1,A1,B1
2,BA0,BB0
3,BA1,BB1


Otra alternativa cuando tenemos índices duplicados es definir un multiíndice

In [96]:
r=pd.concat([a,b],keys=['a','b'])
show_dfs (a, b, r)

Para el acceso a multiíndices utilizamos tuplas con los dos valores del multiíndice, también aplicable a columnas

In [104]:
r.loc[('a',0)]

A    A0
B    B0
Name: (a, 0), dtype: object

## Joins

Las funciones `concat` nos permitían unir DataFrame como nuevas filas o columnas ralizando un alineamiento por índice. Ahora veremos las funciones que nos permiten realizar las uniones de DataFrames de forma similar a como se ralizan en SQL, es decir igualando los valores de las columnas

In [71]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
show_dfs(df1, df2)

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


### Join por el contenido de celdas

La opción más simple es la unión por columnas de dos DataFrames, es un *join 1-1*. Se añade al `df1` la columna *hire_date* del df2. La unión se realiza por los valores de la columna *employee*, que está en los dos dataframes.  
Si sólo indicamos como parámetros los df que se combinan, se combinarán por la columna que se llame igual en ambos DataFrame y utilizando el modo *inner*

In [72]:
df3 = pd.merge(df1,df2)
show_dfs(df1,df2,df3)

En el siguiente caso vemos el join con un DataFrame que tiene menos filas, sólo 3 valores para el group

In [73]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
show_dfs(df3,df4, 
         pd.merge(df3,df4))

En este ejemplo se realiza una unión de 1 a n filas

In [74]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting','Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux','spreadsheets', 'organization']})
show_dfs (df1, df5, 
          pd.merge(df1,df5))

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


Se pueden indicar los campos por los que se hace el join

In [75]:
show_dfs (df1, df2, 
          pd.merge(df1,df2,on="employee"))

Incluso se pueden indicar de forma expresa el campo de cada DataFrames que se va a utilizar en el join

In [105]:
dfname = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                       'salary': [70000, 80000, 120000, 90000]})
show_dfs(df1, dfname, 
           pd.merge(df1,dfname, left_on="employee",right_on="name")
          )

### Joins por valores del índice

Del mismo modo que empleamos los valores de una columan para hacer el join, podemos emplear también los valores de los índices, e incluso, enfrentar el valor de un índice con el de una columna.  
Los siguiente DataFrames tienen definido como índice el nombre del empleado.  
*Para convertir una columna en índice se emplea el método `.set_index()` para realizar la operación contraria `.reset_index()`

In [108]:
df1a=df1.set_index("employee")
df2a=df2.set_index("employee")
show_dfs(df1a,df2a)

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Se le indica con `left_index` y `right_index` que debe utilizar los valores del índice para el join

In [110]:
show_dfs (df1a, df2a, 
            pd.merge (df1a,df2a, 
                      left_index=True, 
                      right_index=True))

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Esta operación es equivalente a la operación `concat`

In [111]:
show_dfs (df1a, df2a, 
            pd.concat([df1a,df2a],axis=1))

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


El método `join()` combinará los DataFrame por los valores sus índices

In [127]:
show_dfs (df1a, df2a, 
          df1a.join(df2a))

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Join combinando índices y columnas

In [128]:
show_dfs (df1a, dfname, 
            pd.merge(df1a, dfname, left_index=True,right_on="name"))

Si quieremos utilizar join tendremos que igualar el contenido de los índices, en este caso se realiza la transformación con `.set_index()` de ese modo igualamos los contenidos de los DataFrames.

In [145]:
show_dfs (df1a, dfname, 
            df1a.join (dfname.set_index("name")))

### Inner, Outer, Left y Right joins

Cualquiera de los joins vistos pueden utilizar uno de los modos indicados, aplicando la propiedad `how`, en el caso de concat también disponemos de facilidad a través de la propiedad `join`.  
Para `merge` y `join` el valor por defecto es *inner*

In [152]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']}
                   ,columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
show_dfs(df6,df7,
         pd.merge(df6, df7),
         caption="inner")

In [153]:
show_dfs(df6,df7,
         pd.merge(df6, df7,how='left'),
        caption="Left")

In [154]:
show_dfs(df6,df7,
         pd.merge(df6, df7,how='right'),
        caption="Right")

In [155]:
show_dfs(df6,df7,
         pd.merge(df6, df7,how='outer'),
        caption="Outer")

En ocasiones podemos tener nombres de columnas repetidos en ambos DataFrames, cuando esto sucede podemos utilizar la propiedad `suffixes=[,]` para marcar los campos de uno y otro dataframe.  
*El método `merge` automáticamente les pondrá el sufijo '_x' e '_y' para indentificarlos

In [163]:
a=make_df()
b=make_df()
show_dfs(a,b,pd.merge(a,b,on='A',suffixes=["_de_A","_de_B"]))